<div style="position: relative; text-align: center;">
  <img src="imagenes/portada.png" alt="INE" width="50%">
</div> <br><br>


<p style="text-align: center; font-size: 20px;"><u>ÍNDICE</u></p>

<span style="font-size: 20px;">

1. **Introducción**

2. **Importación de paquetes**<br>

3. **Funciones**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3.1. Función de limpieza<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3.2. Funcion de inicialización<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3.3. Función de extracción de información<br>

5.  **Proceso de extracción de información**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4.1. Inicializo variables<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4.2. Extracción de información<br>

6. **Comprobación de resultados**<br>

</span>

# 1. Introducción

<span style="font-size: 20px;">

El presente proyecto tratará de extraer información relevante de faturas de luz. Para ello se presentan diferentes modelos de facturas en formato PDF que, mediante el método empleado de extracción de información, esta se guardará en archivos con formato JSON.

El método empleado para la extracción de información ha sido la utilización de un LLM junto con la librería Kor. Esta librería nos permite crear un esquema(*véase Apéndice 1.*) para especificar que información debe ser extraida, generando un prompt que mandará a nuestro LLM.
Los archivos PDF se han leido con la librería Pypdf y el texto se ha limpiado bajo criterio personal, pudiendo mejorarse.

Este método puede implementarse utilizando LLM's open source como LLaMa 3, haciendo un finetuning(*véase Apéndice 2.*) con los datos de entrenamiento y aplicando el mismo procedimiento. No se ha realizado en este caso por carecer de procesamiento suficiente, por lo que se ha optado por el modelo 'gpt-3.5-turbo' de OPENAI.

Se han comparado los resultados obtenidos entre el modelo 'gpt-3.5-turbo' y el modelo LLaMa 3-70b(a traves de GROQ), obteniendo prácticamente identicos resultados.

</span>

# 2. Importación de paquetes

In [2]:
import os
import glob
import json
import pickle
from tqdm import tqdm

import pandas as pd

import re
from pypdf import PdfReader
from langchain.chat_models import ChatOpenAI
from langchain_groq import ChatGroq
from typing import List, Optional
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text

from dotenv import load_dotenv

In [68]:
load_dotenv()

True

# 3. Funciones

## 3.1. Función de limpieza

In [67]:
def bill_cleaner(path):

    """
    Función que devuelve el texto procesado de una factura.

    Input:
        - path(str): Ruta de la factura.pdf
    
    Output:
        - texto_limpio (str)
    
    """

    factura = PdfReader(path)
    
    texto_factura = ""
    for pagina in factura.pages:
        texto_factura += pagina.extract_text()
    
    # Elimino hiperlinks:
    texto_limpio = re.sub(r'\b(?:http://|https://|www\.)?\S+(?:-|\s)?\S*?(?:\.com|\.es)\b', "", texto_factura).strip()
    
    # Elimino conjuntos de puntos mayores a 1:
    texto_limpio = re.sub(r'\.{2,}', "", texto_limpio).strip()
    
    # Elimino espacios multiples en blanco y saltos de linea:
    texto_limpio = re.sub(r"\s+", " ", texto_limpio)

    return texto_limpio

## 3.2. Funcion de inicialización

In [69]:
def inicializar():

    data_path = input("Introduce la ruta donde se encuentran las facturas de las que deseas extraer información: ")

    model = input("Selecciona OPENAI(1) o LLaMa(2): ")

    if model == "1":
        nombre = input("Introduce el nombre asociado a la API key de OPENAI que aparece en tu archivo .ENV: ")
        api_key = os.getenv(nombre)
    if model == "2":
        nombre = input("Introduce el nombre asociado a la API key de GROQ que aparece en tu archivo .ENV: ")
        api_key = os.getenv(nombre)

    return data_path, model, api_key

## 3.3. Función de extracción de información

In [104]:
def information_extractor(path, model, api_key):

    """
    Función que extrae la información requerida de una factura de luz guardandola en formato JSON.

    Input:
        - path(str): Ruta donde se alojan las factura en formato PDF
        - api_key(str): La api key de OPENAI o GROQ
    
    Output:
        - archivo JSON con la información requerida de la factura.
    
    """
    
    # Defino el LLM:
    if model == "1":
        llm = ChatOpenAI(
            model_name="gpt-3.5-turbo",
            temperature=0,
            openai_api_key= api_key)
        
    if model == "2":
        llm = ChatGroq(model="llama3-70b-8192",
                       groq_api_key= api_key)   


    # Cargo el esquema:
    with open("utils/schema.pkl", "rb") as f:
        schema = pickle.load(f)

    # Cargo cadena
    chain = create_extraction_chain(llm, schema, encoder_or_encoder_class="json")
    
    
    
    # Busco las facturas en PDF en la ruta proporcionada:
    
    facturas = glob.glob(os.path.join(path, "*.pdf"))

    # Obtengo la información requerida de cada factura:
    
    if not os.path.exists("extracted_information"):
        os.makedirs("extracted_information")
    
    barra_progreso = tqdm(total= len(facturas), desc="Progreso", unit="archivo")
    
    for path_factura in facturas:
        file_name = path_factura.split("\\")[1].split(".")[0] + ".json"
        extracted_information_path = "extracted_information\\"

        ###################################### Limpieza texto ######################################
        
        texto_factura = bill_cleaner(path_factura)

        ###################################### Extracción de información requerida ######################################
        
        informacion_factura = chain.invoke(input= texto_factura)["text"]["data"]["informacion_factura"][0]

        ###################################### Guardo en formato JSON la informacion ######################################
        with open(extracted_information_path + file_name, "w") as json_file:
            json.dump(informacion_factura, json_file, indent=4)

        
        barra_progreso.update(1)

    barra_progreso.close()

    return extracted_information_path

## 3.4. Función para el cálculo de la distancia de Levenshtein

In [72]:
def distance(str1, str2):
    '''
    Función para el cálculo de la distancia de Levenshtein.
    
    Input:
        - str1(str): String de la predicción.
        - str2(str): String del test.
    
    Output:
        - Distancia de Levenshtein.
    '''
    
    d=dict()
    for i in range(len(str1)+1):
      d[i]=dict()
      d[i][0]=i
    for i in range(len(str2)+1):
      d[0][i] = i
    for i in range(1, len(str1)+1):
      for j in range(1, len(str2)+1):
         d[i][j] = min(d[i][j-1]+1, d[i-1][j]+1, d[i-1][j-1]+(not str1[i-1] == str2[j-1]))
    return d[len(str1)][len(str2)]

## 3.5. Función para el cálculo del score

In [101]:
def score(json_test, json_predicho):
    '''
    Función para calcular el score entre dos archivos JSON, uno es de test y el otro el predicho.

    Input:
        - json_test(JSON): Archivo JSON de la factura test.
        - json_predicho(JSON): Archivo JSON de la factura predicha.
    
    Output:
        - Score entre la información test y la predicha.
    
    '''
    
    distancias = []
    for key in list(json_test.keys()):
        str_1 = str(json_predicho[key])
        str_2 = str(json_test[key])

        if len(str_2) == 0:
            distancia = 0

        else:
            distancia = 1 - (distance(str_1, str_2)/len(str_2))
            
        distancias.append(distancia)    
    
    score = (1/len(json_test.keys()))*sum(distancias)
    
    return score    

## 3.6. Función para el cálculo del score total

In [103]:
def score_total(path_test, path_pred):
    '''
    Función que devuelve la media de los scores obtenidos para cada documento

    Input:
        - path_test(str): Ruta donde se alojan los archivos JSON test.
        - path_pred(str): Ruta donde se alojan los archivos JSON predichos.
    
    Output:
        - Media Score.
    
    '''
    
    json_predichos = glob.glob(os.path.join(path_pred, "*.json"))

    archivos_pred = []
    for i in json_predichos:
        archivo = i.split("\\")[1]
        archivos_pred.append(archivo)

    json_tests = [path_test + archivo for archivo in archivos_pred]

    scores = []
    for test, pred in zip(json_tests, json_predichos):
        
        with open(test, "r", encoding= "utf-8") as archivo_test:
            factura_test = json.load(archivo_test)
            
        with open(pred, "r", encoding= "utf-8") as archivo_pred:
            factura_pred = json.load(archivo_pred)
    
        scr = score(factura_test, factura_pred)
        scores.append(scr)
    
    media = round(sum(scores)/len(scores), 3)
    
    return media    

# 4. Proceso de extracción de información

## 4.1. Inicializo variables

<span style="font-size:larger;">

* **data_path**: Es la carperta donde se encuentran las facturas en pdf
* **openai_api_key**: La clave de la API de OPENAI en el archivo .env

Ejemplo:

- <u>*Usando gpt-3.5-turbo*</u>:

```api_key= os.getenv("OPENAI_API_KEY")
data_path = "data/"
```

- <u>*Usando LLaMa 3-70b*</u>:

```
api_key= os.getenv("GROQ_API_KEY")
data_path = "data/"
```

<br>

<u>**Ejecuta la siguiente linea de código para inicializar las variables y comenzar con el proceso de extracción de información**</u>:

   
</span>

In [81]:
data_path, model, api_key = inicializar()

Introduce la ruta donde se encuentran las facturas de las que deseas extraer información:  data/
Selecciona OPENAI(1) o LLaMa(2):  1
Introduce el nombre asociado a la API key de OPENAI que aparece en tu archivo .ENV:  OPENAI_API_KEY


## 4.2. Extracción de información

<span style="font-size:larger;">

La <u>**siguiente linea de código**</u> ejecutará la función para <u>**extraer información**</u> de las facturas, creando la carpeta *extracted_information* donde se guardará la información de cada una de las faturas en formato JSON, llevando el mismo nombre de la factura correspondiente.

</span>

In [82]:
extracted_information_path = information_extractor(data_path, model, api_key)

C:\Users\regue\conda_ENV\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
Progreso: 100%|██████████| 3/3 [00:16<00:00,  5.42s/archivo]


# 5. Comprobación de resultados
<span style="font-size:larger;">
Los resultados se han comprobado mediante una media de una métrica basada en la distancia de Levenshtein de todos los campos de todos los documentos, y se expresará en porcentaje.

Se requieren los siguientes campos para comprobar el score obtenido:

* **data_path**: Es la carperta donde se encuentran los archivos JSON de las facturas test
* **extracted_information_path**: Es la carperta donde se encuentran los archivos JSON de las facturas predichas
</span>

In [43]:
score_total(data_path, extracted_information_path)

0.935